In [1]:
from kaggle_handler import handler
import numpy as np
import pandas as pd

In [2]:
import ast
import pickle
import joblib
import os

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler

In [4]:
Assets = handler(data_set='tmdb/tmdb-movie-metadata')

Directory 'Assets' already exists.
Datasets already exist in Assets folder
['tmdb_5000_movies.csv', 'tmdb_5000_credits.csv']
 Change Add_more parameter to download more datasets


In [5]:
Assets

['tmdb_5000_movies.csv', 'tmdb_5000_credits.csv']

In [6]:
movies = pd.read_csv('Assets/tmdb_5000_movies.csv')
credits = pd.read_csv('Assets/tmdb_5000_credits.csv') 

In [7]:
movies.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


In [8]:
movies.shape

(4803, 20)

In [9]:
credits.head(2)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [10]:
movies = movies.merge(credits,on='title')

In [11]:
movies.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [12]:
movies = movies[['movie_id','title','overview','production_companies','genres','keywords','cast','crew','popularity','revenue','vote_average','vote_count']]

In [13]:
movies.dropna(inplace=True)

In [14]:
movies_objs = movies[['movie_id','title','overview','production_companies','genres','keywords','cast','crew']].copy()

In [15]:
movies_ints = movies[['popularity','revenue','vote_average','vote_count']].copy()

In [16]:
movies_objs.head(2)

,movie_id,title,overview,production_companies,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [17]:
movies_ints.head(2)

,popularity,revenue,vote_average,vote_count
0,150.437577,2787965087,7.2,11800
1,139.082615,961000000,6.9,4500


In [18]:
def convert(text):
    L = []
    for i in ast.literal_eval(text):
        L.append(i['name']) 
    return L 

In [19]:
movies_objs['production_companies'] = movies_objs['production_companies'].apply(convert)
movies_objs.head(2)

,movie_id,title,overview,production_companies,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Ingenious Film Partners, Twentieth Century Fo...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Walt Disney Pictures, Jerry Bruckheimer Films...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [20]:
movies_objs['genres'] = movies_objs['genres'].apply(convert)
movies_objs.head(2)

,movie_id,title,overview,production_companies,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Ingenious Film Partners, Twentieth Century Fo...","[Action, Adventure, Fantasy, Science Fiction]","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Walt Disney Pictures, Jerry Bruckheimer Films...","[Adventure, Fantasy, Action]","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [21]:
movies_objs['keywords'] = movies_objs['keywords'].apply(convert)
movies_objs.head(2)

,movie_id,title,overview,production_companies,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Ingenious Film Partners, Twentieth Century Fo...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Walt Disney Pictures, Jerry Bruckheimer Films...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [22]:
def convert3(text):
    L = []
    counter = 0
    for i in ast.literal_eval(text):
        if counter < 3:
            L.append(i['name'])
        counter+=1
    return L 

In [23]:
movies_objs['cast'] = movies_objs['cast'].apply(convert)
movies_objs.head(2)

,movie_id,title,overview,production_companies,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Ingenious Film Partners, Twentieth Century Fo...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weave...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Walt Disney Pictures, Jerry Bruckheimer Films...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley, ...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [24]:
movies_objs['cast'] = movies_objs['cast'].apply(lambda x:x[0:6])

In [25]:
def fetch_director(text):
    L = []
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            L.append(i['name'])
    return L 

In [26]:
movies_objs['crew'] = movies_objs['crew'].apply(fetch_director)

In [27]:
#movies['overview'] = movies['overview'].apply(lambda x:x.split())
movies_objs.sample(2)

,movie_id,title,overview,production_companies,genres,keywords,cast,crew
3708,65749,Tango,A dangerous love affair inspires a director to...,[Adela Pictures],"[Drama, Foreign, Romance]","[dancer, tango, musical, love]","[Miguel Ángel Solá, Cecilia Narova, Mía Maestr...",[Carlos Saura]
2078,7341,The Rage: Carrie 2,"After the suicide of her only friend, Rachel h...","[United Artists, Red Bank Films]","[Horror, Thriller, Science Fiction]","[suicide, male nudity, female nudity, fire, nu...","[Emily Bergl, Jason London, Amy Irving, J. Smi...",[Katt Shea]


In [28]:
def collapse(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ",""))
    return L1

In [29]:
movies_objs['cast'] = movies_objs['cast'].apply(collapse)
movies_objs['crew'] = movies_objs['crew'].apply(collapse)
movies_objs['genres'] = movies_objs['genres'].apply(collapse)
movies_objs['keywords'] = movies_objs['keywords'].apply(collapse)
movies_objs['production_companies'] = movies_objs['production_companies'].apply(collapse)

In [30]:
movies_objs.head(2)

,movie_id,title,overview,production_companies,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[IngeniousFilmPartners, TwentiethCenturyFoxFil...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver, ...",[JamesCameron]
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[WaltDisneyPictures, JerryBruckheimerFilms, Se...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley, Ste...",[GoreVerbinski]


In [31]:
movies_objs['overview'] = movies_objs['overview'].apply(lambda x:x.split())

In [32]:
movies_objs['tags'] = movies_objs['overview'] + movies_objs['production_companies'] + movies_objs['genres'] + movies_objs['keywords'] + movies_objs['cast'] + movies_objs['crew']

In [33]:
new = movies_objs.drop(columns=['overview','production_companies','genres','keywords','cast','crew']).copy()
#new.head()

In [34]:
new['tags'] = new['tags'].apply(lambda x: " ".join(x))
new.head(2)

,movie_id,title,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."


In [35]:
movies_ints.head(2)

,popularity,revenue,vote_average,vote_count
0,150.437577,2787965087,7.2,11800
1,139.082615,961000000,6.9,4500


In [36]:
scale = MinMaxScaler()
scale.fit(movies_ints)

,feature_range,"(0, ...)"
,copy,True
,clip,False


In [37]:
movie_ints_scaled = scale.transform(movies_ints)

In [38]:
# movie_ints_scaled = pd.DataFrame(movie_ints_scaled, columns=movies_ints.columns)
movie_ints_scaled.shape

(4806, 4)

In [39]:
cv = CountVectorizer(max_features=5000,stop_words='english')

In [40]:
vector = cv.fit_transform(new['tags']).toarray()

In [41]:
vector.shape

(4806, 5000)

In [42]:
vector = np.concat((vector, movie_ints_scaled),axis=1)

In [43]:
similarity = cosine_similarity(vector)

In [44]:
similarity[0]

array([1.        , 0.10268585, 0.08046027, ..., 0.03428798, 0.03558457,
       0.01088055], shape=(4806,))

In [45]:
def recommend(movie):
    index = new[new['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        print(new.iloc[i[0]].title)

In [46]:
recommend('Avatar')

Independence Day
Aliens vs Predator: Requiem
Titan A.E.
Jupiter Ascending
Lifeforce


In [47]:
movie_dict = new.to_dict()

In [48]:
try:
    os.mkdir(f"Models")
    print(f"Directory 'Models' created successfully.")
except FileExistsError:
    print(f"Directory 'Models' already exists.")

Directory 'Models' already exists.


In [49]:
pickle.dump(movie_dict,open('Models/movie_list.pkl','wb'))
pickle.dump(similarity,open('Models/similarity.pl','wb'))

In [50]:
# joblib.dump(similarity,open('Models/similarity.jl','wb'))